In [ ]:
using CairoMakie

import LilGuys as lguys
using Arya

In [ ]:
function sigma_axis(gp; kwargs...) 
	ax = Axis(gp, 
		xlabel=log_r_label,
		ylabel = L"\log \Sigma\ / \textrm{(stars/arcmin^2)}",
        limits=(-1, 2, -2, 2.5)
		;kwargs...
	)

	return fig, ax
end

In [ ]:
prof_expected = lguys.StellarProfile("/astro/dboyea/dwarfs/observations/sculptor/processed/fiducial_sample_profile.toml");

In [ ]:
log_r_label = "log r / arcmin"

In [ ]:
distance = 81

In [ ]:
function plot_finals(models, labels)
    
    fig = Figure()
    sigma_axis(fig[1,1],
    )

	errscatter!(prof_expected.log_r, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		color=:black,
        label="J+24",
	)

    for i in eachindex(models)
    	lines!(models[i].log_r, models[i].log_Sigma, 
    			label=labels[i])
    end

	axislegend()

	fig
end

In [ ]:
function plot_ini_fin(prof_i, prof_f, model)
    
    fig = Figure()
    sigma_axis(fig[1,1],
    )

	errscatter!(prof_expected.log_r, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		color=:black,
        label="J+24",
	)

    lines!(prof_i.log_r, prof_i.log_Sigma, 
            label="initial")

    lines!(prof_f.log_r, prof_f.log_Sigma, 
            label="final")

    x = LinRange(-2, 2.5, 1000)
    r = 10 .^ x
    y = lguys.calc_Σ.(model, r)

    r_scale = lguys.kpc_to_arcmin(1, distance)

    M_scale = prof_expected.M_in[end] / lguys.calc_M_2D(model, 10^prof_expected.log_r_bins[end])

    x = x .+ log10(r_scale)
    y .*= M_scale / r_scale ^ 2

    lines!(x, log10.(y), linestyle=:dot, color=:black, label="expected")
	axislegend()

	fig
end

In [ ]:
using Measurements

In [ ]:
function get_log_Σ_0(prof, χ2_max=1)

    local log_Σ_0, log_Σ_0_err

    pre_filt = isfinite.(prof.log_Sigma)
    log_r = prof.log_r[pre_filt]
    log_Σ = prof.log_Sigma[pre_filt]
    log_Σ_err = prof.log_Sigma_err[pre_filt]
    @assert issorted(log_r)

    for i in eachindex(log_r)
        filt = 1:i
        log_Σ_0 = lguys.mean(log_Σ[filt] .± log_Σ_err[filt])
        log_Σ_0_err = Measurements.uncertainty.(log_Σ_0)
        log_Σ_0 = Measurements.value(log_Σ_0)
        χ2 = @. (log_Σ_0 - log_Σ)^2 / (log_Σ_err + log_Σ_0_err)^2

        χ2_norm = lguys.mean(χ2[filt])

        if χ2_norm .> χ2_max
            @info "max radius of $(log_r[i])"
            break
        end
    end
    
    return log_Σ_0, log_Σ_0_err
end

In [ ]:
get_log_Σ_0(profiles[1])

In [ ]:
function get_normalization(prof_f, prof_expected=prof_expected)
    log_Σ_0_obs, _ = get_log_Σ_0(prof_expected)
    log_Σ_0, _ = get_log_Σ_0(prof_f)

    return log_Σ_0_obs - log_Σ_0
end

## V=32, r=5.9 (the initial model)

In [ ]:
model_dir = "/astro/dboyea/sculptor/orbits/orbit1/1e6_V32_r5.4/stars/"
stars_dir = "/astro/dboyea/sculptor/isolation/1e6/fiducial/stars_ana/"

In [ ]:
models = [
    "exp2d_rs0.07",
    "exp2d_rs0.10",
    "exp2d_rs0.20",
]

In [ ]:
names_i = [model_dir] .* models .* ["_i_profile.toml"]
names_f = [model_dir] .* models .* ["_profile.toml"]
names_profiles = [stars_dir] .* models .* [".toml"]

In [ ]:
profiles = lguys.StellarProfile.(names_f);
profiles_i = lguys.StellarProfile.(names_i);

In [ ]:
profile_an = lguys.StellarProfile("/astro/dboyea/dwarfs/notebooks/anake_test_profile.toml");

In [ ]:
expected = lguys.load_profile.( names_profiles)

In [ ]:
plot_finals([profiles; profile_an], [models; "ananke"])

In [ ]:
plot_finals(profiles_i, models)

In [ ]:
i = 1
plot_ini_fin(profiles_i[i], profiles[i], expected[i])

### King profiles

In [ ]:
models = [
    "king_rs0.10",
    "king_rs0.15",
]

In [ ]:
names_i = [model_dir] .* models .* ["_1_profile.toml"]
names_f = [model_dir] .* models .* ["_profile.toml"]
names_profiles = [stars_dir] .* models .* [".toml"]

In [ ]:
profiles = lguys.StellarProfile.(names_f);
profiles_i = lguys.StellarProfile.(names_i);

In [ ]:
expected = lguys.load_profile.( names_profiles)

In [ ]:
plot_finals(profiles, models)

In [ ]:
plot_finals(profiles_i, models)

In [ ]:
expected[2].R_t

In [ ]:
lguys.kpc_to_arcmin(0.75, 81)

In [ ]:
log10(31)

# Orbit studies

## V=32, r=2.4

In [ ]:
models = [
    "exp2d_rs0.05",
    "exp2d_rs0.10",
    "exp2d_rs0.20",
]

In [ ]:
model_dir = "/astro/dboyea/dwarfs/analysis/sculptor/1e6_V31_r3.2/orbit1/stars/"
stars_dir = "/astro/dboyea/dwarfs/analysis/sculptor/1e6_V31_r3.2/stars/"

names_i = [model_dir] .* models .* ["/initial_profile.toml"]
names_f = [model_dir] .* models .* ["/final_profile.toml"]
names_profiles = [stars_dir] .* models .* ["/profile.toml"]

In [ ]:
profiles_all = lguys.read_structs_from_hdf5("/astro/dboyea/dwarfs/analysis/sculptor/1e6_V31_r3.2/orbit1/stars/exp2d_rs0.10/stellar_profiles.hdf5", lguys.StellarProfile);

In [ ]:
expected = lguys.load_profile.( names_profiles)

In [ ]:
profiles = lguys.StellarProfile.(names_f);

In [ ]:
profiles_i = lguys.StellarProfile.(names_i);

In [ ]:
norms = get_normalization.(profiles)

In [ ]:
[prof.log_Sigma .+= norm for (prof, norm) in zip(profiles, norms)]

In [ ]:
[prof.log_Sigma .+= norm for (prof, norm) in zip(profiles_i, norms)]

In [ ]:
prof_fit = lguys.KingProfile(M=1, R_s=11, R_t=25)

In [ ]:
r_b = 76

In [ ]:
i = 2
plot_ini_fin(profiles_i[i], profiles[i], expected[i])

In [ ]:
sum(prof_expected.Sigma * 2π .* 10 .^ prof_expected.log_r .* diff(10 .^ prof_expected.log_r_bins))

In [ ]:
prof_expected.M_in[end]

In [ ]:
get_log_Σ_0(profiles[1])

In [ ]:
plot_finals(profiles, models)

In [ ]:
fig, ax = FigAxis(
    xlabel = "log r / kpc",
    ylabel = "log Sigma",
    limits=(-3, 1, -5, 2)
)

for (name, prof) in profiles_all
    x = prof.log_r
    y = prof.log_Sigma

    i = parse.(Int64, name)
    
    lines!(x, y, color=i, colorrange=(1, length(profiles_all)))
end

fig